## First, let's install and import all required libraries

In [ ]:
#!pip install -r requirements.txt

In [ ]:
import collections                        #This library adds some extras to the starndard python data structures
import folium                             #Great library for plotting data on maps
import json                               #This library allows us to load and hyndle json files with python
import os                                 #This module is usefull for handling paths and directories
import webbrowser                         #This library allows to hanle the webbrowser from python
import pandas as pd
from folium.plugins import HeatMap
from progressbar import ProgressBar, Bar, ETA, Percentage
from utils import timestampInRange

## Now, let's load and analyze our location history json file 

In [ ]:
data = json.load(open('Location_History_Juan.json'))

In [ ]:
data

In [ ]:
type(data)

In [ ]:
data["locations"]

In [ ]:
df = pd.DataFrame(data["locations"])

In [ ]:
df = df.drop(columns=['activity'])
df

## Now we can extract and tranform the data we need to make our map 

In [ ]:
coordinates = collections.defaultdict(int)

In [ ]:
coordinates

In [ ]:
max_coordinates = (0, 0)

In [ ]:
max_magnitude = 0

In [ ]:
"""Here you can introduce a range of dates for the map. for example date_range=('2000-01-02', '2019-12-31')"""
date_range = (None, None)


"""Here you transfor the coordinates given by google in actual longitude and latitude coordinates and
show the progress using the progress bar widget"""

w = [Bar(), Percentage(), " ", ETA()]
with ProgressBar(max_value=len(data["locations"]), widgets=w) as pb:
    for i, loc in enumerate(data["locations"]):
        # print(i)
        # print(loc)
        if "latitudeE7" not in loc or "longitudeE7" not in loc:
            continue
        coords = (round(loc["latitudeE7"] / 1e7, 6),
                    round(loc["longitudeE7"] / 1e7, 6))
        # print(coords)

        
        """Here you calculate the magnitude for all coordinates within the date range"""
        
        if timestampInRange(loc["timestampMs"], date_range): 
            #print(loc["timestampMs"])
            coordinates[coords] += 1 #these are the magnitude values we will need for the coordinates dictionary
            #print(coordinates[coords])
            
            if coordinates[coords] > max_magnitude:
                max_coordinates = coords
                max_magnitude = coordinates[coords]
                #print(max_coordinates)
                #print(max_magnitude)
        
        """This command updates the progress bar"""
        pb.update(i)

In [ ]:
coordinates

In [ ]:
tilesoptions = ["openstreetmap", "StamenTerrain", "stamentoner",
                "stamenwatercolor", "cartodbpositron", "cartodbdark_matter"]

tiles = tilesoptions[5]
zoom_start = 10
radius = 7
blur = 4
min_opacity = 0.2
max_zoom = 10


In [ ]:
map_data = [(coords[0], coords[1], magnitude)
            for coords, magnitude in coordinates.items()] #coords are keys in the dict. magnitudes are the values

In [ ]:
map_data

In [ ]:
# Generate background map
m = folium.Map(location=max_coordinates,
                zoom_start=zoom_start,
                tiles=tiles)

# Generate heat map
heatmap = HeatMap(map_data,
                    max_val=max_magnitude,
                    min_opacity=min_opacity,
                    radius=radius,
                    blur=blur,
                    max_zoom=max_zoom)

m.add_child(heatmap)

## Finally, we can save our map as an html file and launch it in the browser 

In [ ]:
output_file = tiles + 'heatmapJuan.html'

In [ ]:
m.save(output_file)

In [ ]:
webbrowser.open("file://" + os.path.realpath(output_file))